# Setting up Project 
- makes maps 
- uses Google Api (gmaps)
- GeoJSON objects (NYC)
- excel Files for Data

In [57]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import folium
import gmaps
import gmaps.geojson_geometries
import os
import json
import requests
import pprint as pp

import csv

# Import API key
from config import g_key

### Load in Fixed NYC CSV
- Turn it into a final dictionary with:
* keys = Neighborhood names
* values = Corresponding DataFrame

In [58]:
data = pd.read_csv('checker1.csv')
data.head()

,accommodates,amenities,availability_365,id,latitude,longitude,name,neighbourhood,new neighborhood,number_of_reviews,number_of_reviews_ltm,price,property_type,review_scores_location,review_scores_rating,reviews_per_month,room_type
0,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",279.0,715270.0,40.85956,-73.87067,2 Beds/Queen & Full Beautiful Room 40 minsT.Sq...,Allerton,Allerton-Pelham Gardens,170.0,20.0,39.0,Apartment,9.0,90.0,2.05,Private room
1,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",234.0,755528.0,40.85840,-73.86969,PRIVATE BATH/TONS OF SUNLIGHT/SAFE,Allerton,Allerton-Pelham Gardens,192.0,46.0,49.0,Apartment,9.0,90.0,2.33,Entire home/apt
2,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",253.0,773041.0,40.85914,-73.86979,Nice beautiful room In the Bronx,Allerton,Allerton-Pelham Gardens,191.0,61.0,38.0,Apartment,10.0,88.0,2.36,Private room
3,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,""F...",0.0,18702075.0,40.87309,-73.86326,Clean and large bedroom in a private house,Allerton,Allerton-Pelham Gardens,5.0,0.0,60.0,Apartment,9.0,87.0,0.19,Private room
4,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Free stre...",235.0,20904409.0,40.86144,-73.86427,Luxury Tiny house • Ohka,Allerton,Allerton-Pelham Gardens,101.0,55.0,79.0,Guest suite,10.0,97.0,4.53,Entire home/apt


In [59]:
# LOADING UNIQUE NEW NEIGHBORHOOD VALUES TO LIST
final_city_list = []
big_list = list(data['new neighborhood'])
for row in big_list:
    if row not in final_city_list:
        final_city_list.append(row)
final_city_list

# Create a Dictionary with new neighbourhood DataFrames
final_dict = {}
for city in final_city_list:
    checker = (data['new neighborhood'] == city)
    city_df = data.where(checker)
    city_df = city_df.dropna()
    final_dict[city] = city_df
final_dict

{'Allerton-Pelham Gardens':    accommodates                                          amenities  \
 0           2.0  {TV,Internet,Wifi,"Air conditioning",Kitchen,"...   
 1           3.0  {TV,"Cable TV",Internet,Wifi,"Air conditioning...   
 2           2.0  {TV,"Cable TV",Internet,Wifi,"Air conditioning...   
 3           2.0  {TV,Wifi,"Air conditioning",Kitchen,Heating,"F...   
 4           2.0  {TV,Wifi,"Air conditioning",Kitchen,"Free stre...   
 5           4.0  {TV,"Air conditioning","Smoking allowed","Pets...   
 6           2.0  {TV,Wifi,"Air conditioning",Kitchen,Breakfast,...   
 7           4.0  {TV,Wifi,Kitchen,"Pets allowed","Free street p...   
 8           4.0  {Wifi,"Air conditioning",Kitchen,"Free street ...   
 9           4.0  {TV,"Cable TV",Wifi,"Air conditioning",Kitchen...   
 
    availability_365          id  latitude  longitude  \
 0             279.0    715270.0  40.85956  -73.87067   
 1             234.0    755528.0  40.85840  -73.86969   
 2             253.

In [217]:
#Load GEO JSON MAP
neighborhood_json_list = []

with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    geometry = json.load(f)
    

    for row in geometry['features']:
        city = row['properties']['neighborhood']
        if city not in neighborhood_json_list:
            neighborhood_json_list.append(city)

neighborhood_json_list

['Allerton',
 'Alley Pond Park',
 'Arden Heights',
 'Arlington',
 'Arrochar',
 'Arverne',
 'Astoria',
 'Bath Beach',
 'Battery Park City',
 'Bay Ridge',
 'Bay Terrace',
 'Bay Terrace, Staten Island',
 'Baychester',
 'Bayside',
 'Bayswater',
 'Bedford-Stuyvesant',
 'Belle Harbor',
 'Bellerose',
 'Belmont',
 'Bensonhurst',
 'Bergen Beach',
 'Bloomfield',
 'Boerum Hill',
 'Borough Park',
 'Breezy Point',
 'Briarwood',
 'Brighton Beach',
 'Broad Channel',
 'Bronx Park',
 'Bronxdale',
 'Brooklyn Heights',
 'Brownsville',
 "Bull's Head",
 'Bushwick',
 'Cambria Heights',
 'Canarsie',
 'Carroll Gardens',
 'Castle Hill',
 'Castleton Corners',
 'Central Park',
 'Charleston',
 'Chelsea',
 'Chelsea, Staten Island',
 'Chinatown',
 'City Island',
 'Civic Center',
 'Claremont Village',
 'Clason Point',
 'Clifton',
 'Clinton Hill',
 'Co-op City',
 'Cobble Hill',
 'College Point',
 'Columbia St',
 'Concord',
 'Concourse',
 'Concourse Village',
 'Coney Island',
 'Corona',
 'Country Club',
 'Crotona Park

#### Check to see if Neighborhood Names matches GeoJSON names

In [270]:
# DATA MUNGING TO FIX CITY MAP
final_city_list
neighborhood_json_list
missing_cities = []
extra_json_cities = []


for city in final_city_list:
    if city not in neighborhood_json_list:
        missing_cities.append(city)
        
for city in neighborhood_json_list:
    if city not in final_city_list:
        extra_json_cities.append(city)


#ALPHABETIZE Lists
neighborhood_json_list.sort()
#missing_cities.sort()

checker_df = pd.DataFrame({'missing cities': pd.Series(missing_cities), 
              'json list': pd.Series(neighborhood_json_list),
             'extra json to use': pd.Series(extra_json_cities)})

checker_df



,missing cities,json list,extra json to use
0,Allerton-Pelham Gardens,Allerton,Allerton
1,Annadale-Huguenot-Prince's Bay-Eltingville,Alley Pond Park,Alley Pond Park
2,NaN,Arden Heights,Arden Heights
3,Battery Park City-Lower Manhattan,Arlington,Arlington
4,Bayside-Bayside Hills,Arrochar,Arrochar
5,Bedford,Arverne,Arverne
6,Bedford Park-Fordham North,Astoria,Astoria
7,Bensonhurst West,Bath Beach,Bath Beach
8,Brooklyn Heights-Cobble Hill,Battery Park City,Battery Park City
9,Bushwick South,Bay Ridge,Bay Ridge


In [434]:
print(neighborhood_json_list.index("Hell's Kitchen"))


121


In [236]:
# Function to Change Names
def change_name(self,key,new_key):
    value = self[key]
    self.pop(key)
    self[new_key] = value

In [237]:
# Function to Update Dictionary
def title_name_change(self,old_key, new_key):
    change_name(self,old_key,new_key)
    dataframe = self[new_key]
    dataframe['new neighborhood'] = new_key

In [327]:
def change_neighborhood_name(self, key):
    dataframe = self[key]
    dataframe['new neighborhood'] = key

In [441]:
title_name_change(copy, "Crown Heights North", "Crown Heights")

In [449]:
checker = copy["DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill"]
add_df = checker[checker['neighbourhood'] == 'Downtown Brooklyn']
add_df 
copy['Downtown Brooklyn'] = add_df

In [456]:
# copy["Cobble Hill"]['new neighborhood'] = "Cobble Hill"
#list(copy["DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill"]['neighbourhood'])

copy['D']

,accommodates,amenities,availability_365,id,latitude,longitude,name,neighbourhood,new neighborhood,number_of_reviews,number_of_reviews_ltm,price,property_type,review_scores_location,review_scores_rating,reviews_per_month,room_type
11860,4.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",94.0,765569.0,40.69594,-73.98375,"Brand New, Boutique Brooklyn Condo",Downtown Brooklyn,Downtown Brooklyn,64.0,4.0,175.0,Apartment,10.0,97.0,0.79,Entire home/apt
11861,4.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,821135.0,40.69379,-73.98525,Bright Loft One Stop from Soho,Downtown Brooklyn,Downtown Brooklyn,41.0,3.0,170.0,Loft,9.0,97.0,0.54,Entire home/apt
11862,4.0,"{TV,Wifi,""Air conditioning"",Kitchen,Breakfast,...",0.0,1408406.0,40.69648,-73.98261,"Beautiful, spacious apartment - with a cute cat!",Downtown Brooklyn,Downtown Brooklyn,11.0,4.0,162.0,Apartment,9.0,98.0,0.43,Entire home/apt
11863,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,2202858.0,40.69093,-73.97936,Gorgeous Ft. Greene apt amazing vu,Downtown Brooklyn,Downtown Brooklyn,4.0,0.0,200.0,Apartment,10.0,100.0,0.06,Entire home/apt
11864,2.0,"{Wifi,""Air conditioning"",Kitchen,""Pets allowed...",0.0,2744106.0,40.69226,-73.98056,BKLYN private room w/ private bath!,Downtown Brooklyn,Downtown Brooklyn,12.0,0.0,95.0,Apartment,9.0,93.0,0.19,Private room
11865,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",92.0,3840541.0,40.69054,-73.99039,Spacious Modern Loft in Cobble Hill,Downtown Brooklyn,Downtown Brooklyn,71.0,24.0,150.0,Loft,10.0,99.0,1.38,Entire home/apt
11866,2.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Brea...",307.0,4866426.0,40.68983,-73.99114,Private Room in Prime Brooklyn Spot,Downtown Brooklyn,Downtown Brooklyn,168.0,36.0,80.0,Apartment,10.0,97.0,3.05,Private room
11867,1.0,"{Internet,Wifi,""Air conditioning"",Pool,Kitchen...",0.0,5193916.0,40.69605,-73.98207,Private room available in Dumbo Brooklyn,Downtown Brooklyn,Downtown Brooklyn,3.0,0.0,45.0,Apartment,9.0,93.0,0.08,Private room
11868,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,5725154.0,40.68927,-73.98423,DNTWN BK / Boerum Hill Sky Loft,Downtown Brooklyn,Downtown Brooklyn,2.0,0.0,150.0,Apartment,10.0,100.0,0.04,Entire home/apt
11869,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",0.0,7501154.0,40.68852,-73.98697,Charming sunny studio - Boerum Hill,Downtown Brooklyn,Downtown Brooklyn,12.0,0.0,119.0,Apartment,10.0,97.0,0.25,Entire home/apt


In [442]:
copy_city_list = []
for k,v in copy.items():
    if k not in copy_city_list:
        copy_city_list.append(k)
        

In [443]:
# DATA MUNGING TO FIX CITY MAP
copy_city_list
neighborhood_json_list
missing_cities = []
extra_json_cities = []


for city in copy_city_list:
    if city not in neighborhood_json_list:
        missing_cities.append(city)
        
for city in neighborhood_json_list:
    if city not in copy_city_list:
        extra_json_cities.append(city)


#ALPHABETIZE Lists
neighborhood_json_list.sort()
missing_cities.sort()

checker_df = pd.DataFrame({'missing cities': pd.Series(missing_cities), 
              'json list': pd.Series(neighborhood_json_list),
             'extra json to use': pd.Series(extra_json_cities)})

print('WORK LEFT TO DO', len(missing_cities))

checker_df


WORK LEFT TO DO 36


,missing cities,json list,extra json to use
0,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Allerton,Alley Pond Park
1,East Concourse-Concourse Village,Alley Pond Park,Arden Heights
2,East Flatbush-Farragut,Arden Heights,Arlington
3,East Harlem South,Arlington,Arrochar
4,Eastchester-Edenwald-Baychester,Arrochar,Arverne
5,Fresh Meadows-Utopia,Arverne,Astoria
6,Georgetown-Marine Park-Bergen Beach-Mill Basin,Astoria,Bath Beach
7,Grasmere-Arrochar-Ft. Wadsworth,Bath Beach,Bay Ridge
8,Grymes Hill-Clifton-Fox Hills,Battery Park City,Bay Terrace
9,Hudson Yards-Chelsea-Flatiron-Union Square,Bay Ridge,"Bay Terrace, Staten Island"


In [428]:
neighborhood_json_list

['Allerton',
 'Alley Pond Park',
 'Arden Heights',
 'Arlington',
 'Arrochar',
 'Arverne',
 'Astoria',
 'Bath Beach',
 'Battery Park City',
 'Bay Ridge',
 'Bay Terrace',
 'Bay Terrace, Staten Island',
 'Baychester',
 'Bayside',
 'Bayswater',
 'Bedford-Stuyvesant',
 'Belle Harbor',
 'Bellerose',
 'Belmont',
 'Bensonhurst',
 'Bergen Beach',
 'Bloomfield',
 'Boerum Hill',
 'Borough Park',
 'Breezy Point',
 'Briarwood',
 'Brighton Beach',
 'Broad Channel',
 'Bronx Park',
 'Bronxdale',
 'Brooklyn Heights',
 'Brownsville',
 "Bull's Head",
 'Bushwick',
 'Cambria Heights',
 'Canarsie',
 'Carroll Gardens',
 'Castle Hill',
 'Castleton Corners',
 'Central Park',
 'Charleston',
 'Chelsea',
 'Chelsea, Staten Island',
 'Chinatown',
 'City Island',
 'Civic Center',
 'Claremont Village',
 'Clason Point',
 'Clifton',
 'Clinton Hill',
 'Co-op City',
 'Cobble Hill',
 'College Point',
 'Columbia St',
 'Concord',
 'Concourse',
 'Concourse Village',
 'Coney Island',
 'Corona',
 'Country Club',
 'Crotona Park

In [349]:

#data = pd.read_json('resources/nyc-neighborhoods.geo.json')
#data

with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    nyc_neighborhoods = json.load(f)
#nyc_neighborhoods['features'][0]

fig = gmaps.figure()
geojson_layer = gmaps.geojson_layer(nyc_neighborhoods)
fig.add_layer(geojson_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# new_york_coordinates = (40.75, -74.00)

# m = folium.Map(location=new_york_coordinates, zoom_start = 9)
# m